In [1]:
import re
import os
import numpy as np
import pandas as pd
import warnings
from pymatgen.core import Structure
from automatminer import AutoFeaturizer

warnings.filterwarnings('ignore')

In [ ]:
data_form = pd.read_csv("../data") 
data_form.head()

In [6]:
cif_dir = './cifs/cifs/'
cif_files= os.listdir(cif_dir)
c=[]
for f in cif_files:
    if f.endswith(".cif"):
        f = re.sub("\D", "", f)
        c.append(f)
c_1 = sorted(list(map(int, c)))
new_cif_files = [(str(i)+'.cif') for i in c_1]
cif_paths = [os.path.join(cif_dir, cif) for cif in new_cif_files]

structures = [Structure.from_file(cif) for cif in cif_paths]
structure_df = pd.DataFrame({"structure": structures})

structure_df.head()

In [ ]:
data_form['structure'] = structure_df
data = data_form
data.head()

In [ ]:
df = data.loc[:, ['structure', 'target']]
df.head()

In [ ]:
featurizer = AutoFeaturizer(preset="express")

featurizer.fit(df, target="target")

featurizer_df = featurizer.transform(df, target="target")

In [ ]:
featurizer_df.head()

In [ ]:
df = featurizer_df.drop(['compound possible', 'is_centrosymmetric'], axis=1)
df

In [28]:
df['crystal_system'] = df['crystal_system'].map({
    'hexagonal' : 2, 
    'cubic' : 1, 
    'trigonal' : 7, 
    'tetragonal' : 4, 
    'orthorhombic' : 5, 
    'triclinic' : 3, 
    'monoclinic' : 6
}).astype(int)

In [30]:
df.to_csv('FP.csv', index=False)